# Movie Lens MLOps with TFX

The goal of a recommender system for Movie Lens is to learn to accurately predict a user's ratings
in order to be able to suggest movies to the user that they would like and have not seen.
See also the section below called "Linking to business value".

The Movie Lens data is extracted, analyzed, processed and stored and registered with an MLMetadata registry.  Regression models are trained and tuned
to predict the ratings.  The best models are evaluated, validated, and deployed to storage, MLMD registry, and/or serving endpoints.  The deployed models are monitored for model degradation and anomalies.  The live 
incoming data is also monitored for shifts in distributions.
The model validation involves testing, including A/B testing if possible and manual confirmation that the
model aligns with business objectives.
All of these steps are called machine learning operations (MLOps) and are automated when possible.
There are iterative and continuous loops in segments of a total MLOps pipeline or pipelines.

## Using Tensorflow's TFX for MLOps
where MLOps is Continuous delivery and automation pipelines in machine learning.

This is a local, non-Kaggle notebook in which TFX 1.16.0 and python 3.10 and the compatible versions of other libraries are installed in a virtual environment that this notebook is running in.  (The Kaggle notebook can run the pipelines in a virtual environment, but cannot display the results using the TFDV or TFMA libraries at this time due to python version conflicts.)

The source code for this notebook is [at github, as recommender_systems](https://github.com/nking/recommender_systems)

The model being tuned and trained is a multi-tower model, 2 towers specifically, and is also known as a bi-encoder.

The use of vector databases and re-ranker is discussed in the other Kaggle notebook.
(Add reference here).

paths are relative to the github repository directory, "recommender_systems"

In [1]:
import shutil

from tfx.orchestration import metadata

import tensorflow_transform as tft

from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

import sys
import os
sys.path.append(os.path.join(os.getcwd(), "src/test/python/movie_lens_tfx"))
sys.path.append(os.path.join(os.getcwd(), "src/main/python/movie_lens_tfx"))

from helper import *
from movie_lens_tfx.PipelineComponentsFactory import *
from movie_lens_tfx.tune_train_movie_lens import *

from absl import logging
tf.get_logger().propagate = False
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

2025-11-17 17:00:32.162331: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:00:32.165520: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:00:32.209929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.a

## EDA on the raw data

### w/ Polars and Plotly express
output is written to bin/local_notebook/images/

%run src/main/python/eda/eda_raw.py

the generated images aren't plotted here, but similar plots are shown via TFDV below.

### Run data pre-processing on full dataset to get the transformed data

In [2]:
infiles_dict_ser, output_config_ser, split_names = get_test_data(use_small=False)
user_id_max = 6040
movie_id_max = 3952
n_genres = N_GENRES
n_age_groups = N_AGE_GROUPS
n_occupations = 21
MIN_EVAL_SIZE = 50 #make this larger for production pipeline

BATCH_SIZE = 64
NUM_EPOCHS = 20

test_num = "1"
    
PIPELINE_NAME = 'TestPipelines'
output_data_dir = os.path.join(get_bin_dir(), "local_notebook", test_num)
PIPELINE_ROOT = os.path.join(output_data_dir, PIPELINE_NAME)

# remove results from previous test runs:
try:
  print(f"removing: {PIPELINE_ROOT}")
  shutil.rmtree(PIPELINE_ROOT)
except OSError as e:
  pass
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'tfx_metadata',
                             'metadata.db')
os.makedirs(os.path.join(PIPELINE_ROOT, 'tfx_metadata'),
            exist_ok=True)

ENABLE_CACHE = True

# metadata_connection_config = metadata_store_pb2.ConnectionConfig()
# metadata_connection_config.sqlite.SetInParent()
# metadata_connection = metadata.Metadata(metadata_connection_config)
metadata_connection_config = metadata.sqlite_metadata_connection_config(
  METADATA_PATH)

store = metadata_store.MetadataStore(metadata_connection_config)

if get_kaggle():
  tr_dir = "/kaggle/working/"
else:
  tr_dir = os.path.join(get_project_dir(), "src/main/python/movie_lens_tfx")

serving_model_dir = os.path.join(PIPELINE_ROOT, 'serving_model')
output_parquet_path = os.path.join(PIPELINE_ROOT, "transformed_parquet")

# for the custom ingestion component, the apache beam pipeline needs to be able to
# find the sibling scripts it imports.
# 2 solutions: (1) create a tar archive and use --extra_package in pipeline args
# or (2) use setup.py and --setup_file in pipeline args.

SETUP_FILE_PATH = os.path.abspath('setup.py')
beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0',
  f'--setup_file={SETUP_FILE_PATH}',
  #f'--extra_package={ingest_tar_file}'
]

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
removing: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines


In [3]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
import logging
logging.getLogger().setLevel(logging.WARNING)
from absl import logging
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

context = InteractiveContext(pipeline_name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args
)

factory = PipelineComponentsFactory(
    num_examples=1000209, infiles_dict_ser=infiles_dict_ser,
    output_config_ser=output_config_ser, transform_dir=tr_dir,
    user_id_max=user_id_max, movie_id_max=movie_id_max,
    n_genres=n_genres, n_age_groups=n_age_groups,
    min_eval_size=MIN_EVAL_SIZE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, device="CPU",
    serving_model_dir=serving_model_dir, output_parquet_path=output_parquet_path)

components = factory.build_components(PIPELINE_TYPE.PREPROCESSING)

for i, component in enumerate(components):
    #print(f"i={i} component={component}")
    context.run(component)

print(f'done pre-processing data')

type=PIPELINE_TYPE.PREPROCESSING
module_file=/home/nichole/projects/github/recommender_systems/src/main/python/movie_lens_tfx/transform_movie_lens.py
i=0 component=MovieLensExampleGen(spec: <tfx.dsl.component.experimental.utils.MovieLensExampleGen_Spec object at 0x70fc89a3b280>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x70fc89a3b4f0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: MovieLensExampleGen, inputs: {}, outputs: {'output_examples': OutputChannel(artifact_type=Examples, producer_component_id=MovieLensExampleGen, output_key=output_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)})


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/goo

i=1 component=StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x70fc89a398d0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x70fc89a3beb0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=MovieLensExampleGen, output_key=output_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}, outputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)})


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/goo

i=2 component=SchemaGen(spec: <tfx.types.standard_component_specs.SchemaGenSpec object at 0x70fc89a30490>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x70fc89a307f0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: SchemaGen, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}, outputs: {'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)})
i=3 component=ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x70fc89a31030>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x70fc89a31120>, driver_class: <class 'tfx.dsl.components.

...

INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+fb13acbbc55a3dc5e4140287ddbb8a25fbbb448ebb0d37bb248f454000152047-py3-none-any.whl'.
DEBUG:absl:Inputs to executor.Transform function: {'disable_statistics': False, 'schema_path': '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/SchemaGen/schema/3/schema.pbtxt', 'examples_data_format': 6, 'data_view_uri': None, 'analyze_data_paths': ['/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/MovieLensExampleGen/output_examples/1/Split-train/*'], 'analyze_paths_file_formats': ['tfrecords_gzip'], 'transform_data_paths': ['/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/MovieLensExampleGen/output_examples/1/Split-test/*', '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/MovieLensExampleGen/output_examp

DEBUG:absl:inputs['genres']=Tensor("inputs_copy:0", shape=(None, 1), dtype=string)
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
DEBUG:absl:inputs['genres']=Tensor("inputs_copy:0", shape=(None, 1), dtype=string)
DEBUG:absl:inputs['genres']=Tensor("inputs_copy:0", shape=(None, 1), dtype=string)
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Featu

i=5 component=FromTFRecordToParquet(spec: <tfx.dsl.component.experimental.utils.FromTFRecordToParquet_Spec object at 0x70fc89a325c0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x70fc89a324a0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: FromTFRecordToParquet, inputs: {'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}, outputs: {})


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/goo

done pre-processing data


## EDA on the transformed data

### using Polars, Plotly.express 

this can take an hour on a single COTS

%run src/main/python/eda/eda_transformed.py

Some of the images are shown here from a former saved run.

You can see a clear correlation between rating and movie_id, then less
so between rating and age and rating and occupation.

<img src="src/test/resources/train_dist_corr_heatmap.png">

You can see many co-occurrences between movie genres.

<img src="src/test/resources/train_genre_cooccurence_rating_5_heatmap.png">

You can see that Drama, then Comedy, then Action are the most popular movie genres.

<img src="src/test/resources/train_genres_rating_5.png">


### using Pandas, Pyspark MLLIB FPGrowth

This does a market basket analysis with movie_ids.

If you want the PrefixSpan plots also, set
PLOT_PREFIXSPAN = True
but beware that the script will take much longer to run.

In [4]:
PLOT_PREFIXSPAN=False

%run src/main/python/eda/eda_transformed_pyspark_mllib.py

Some of the images are shown here from a former saved run.

You can see that many association rules can be derived from the data.
<img src="src/test/resources/train_movies_assoc_rules_rating_5.png">

and that there are many frequent itemsets.
<img src="src/test/resources/train_movies_itemsets_rating_5.png">

<img src="src/test/resources/train_movies_itemsets_rating_5_2.png">

The results show that neural network models will have good material to exploit.

The PrefixScan model can be enabled in the script to provide images to explore sequences
for actions or as precursor to Sequential DNN models.

### Data and Concept Drift
After exploring the data inputs to the model, we want to define monitoring
for data and concept shifts.

let X = features

let Y = targets

Data shift is a change in the joint distribution, P(X, Y). 

Using the probability product rule, we can explore 4 causes for the
simplest changes in P(X,Y)
$$ P(X, Y) = P(Y, X) = P(X|Y)P(Y) = P(Y|X)P(X) $$

We can look for changes in one member in the following pairs at any time
(one is simpler than exploring more than 1 member changing at same time):
$$ P(X|Y) * P(Y) $$
$$ P(Y|X) * P(X) $$

* Covariate shift:
  $$ P(X) changed,  P(Y|X) unchanged $$
  Distr of model inputs changes.
* Label shift:
  $$ P(Y) changed,  P(X|Y) unchanged $$
  Distr of model outputs changes, but for any given output, the input distribution stays the same.
  
* Concept shift:
  $$ P(X) unchanged,  P(Y|X) changed $$
* Manifestation shift:
  $$ P(Y) unchanged,  P(X|Y) changed $$

[see more at NannyML](https://www.nannyml.com/blog/concept-drift)

if the train, eval, and test split are random, then one could use
the maximum differences in their distributions as a lower limit on 
the estimate for stochastic error.  A trigger for data drift should
then be GEQ about 3 times that stochastic error.

The previous data doesn't exist yet for this project, but one could
either download another movie-lens dataset of different time period,
or split this 1M dataset by a timestamp ordering into 2 partitions.

More data is available at [GroupLens](https://files.grouplens.org/datasets/movielens/)

### Data [Drift and Skew using TFDV](https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic)
* Drift
  Drift detection is supported for categorical features and between consecutive spans of data
  (i.e., between span N and span N+1), such as between different days of training data.
    * L-infinity distance
    * alerts when drift is higher than threshold distance
* Skew
    * data - schema skew
      occurs when the training and serving data do not conform to the same schema.
      Any expected deviations between the two (such as the label feature being only present in the training data but not in serving) should be specified through environments field in the schema.
    * feature skew
      occurs when the training and serving feature values are different
      This can happen when:
        * A data source of features is modified between training and serving time
        * A difference in logic for generating between training and serving.
    * distribution skew
      occurs when the training and serving distributions are significantly different.
      some key causes:
        * different code or different data sources to generate the training dataset
        * a faulty sampling mechanism that chooses a non-representative subsample of the serving data to train on.


TODO: finish adding to this below with the components measuring them.

### using TFDV to look at the raw data

In [5]:
from tfx.dsl.io import fileio
from tfx.orchestration import metadata
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import anomalies_pb2, schema_pb2
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_data_validation as tfdv

In [6]:
print("Schema from SchemaGen:")
_list = store.get_artifacts_by_type("Schema")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "SchemaGen" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_path = os.path.join(artifact_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(file_path)
#tfdv.visualize_artifacts(schema)
tfdv.display_schema(schema=schema)

print("ExampleStatistics from StatisticsGen:")
_list = store.get_artifacts_by_type("ExampleStatistics")
print(f'ExampleStatistics count={len(_list)}')
#print(f'ExampleStatistics ={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "StatisticsGen" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
stats_dict = {}
for file_path in file_paths:
    if "test" in file_path:
        #plotting train and eval
        continue
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    if "train" in file_path:
        stats_dict['train'] = stats
    else:
        stats_dict['eval'] = stats
tfdv.visualize_statistics(lhs_statistics=stats_dict['eval'],
    rhs_statistics=stats_dict['train'],
    lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

print("ExampleAnomalies from ExampleValidator:")
_list = store.get_artifacts_by_type("ExampleAnomalies")
#print(f'ExampleAnomalies count={len(_list)}\nlist={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
artifact_uri = None
for artifact in _list:
    if "ExampleValidator" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "SchemaDiff.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)

print("ExampleStatistics from pre-transform stats of Transform:")
_list = store.get_artifacts_by_type("ExampleStatistics")
#print(f'ExampleStatistics={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    

Schema from SchemaGen:
Schema count=3


,Type,Presence,Valency,Domain
Feature name,,,,
'genres',BYTES,required,,-
'gender',STRING,required,,'gender'
'age',INT,required,,-
'movie_id',INT,required,,-
'occupation',INT,required,,-
'rating',INT,required,,-
'timestamp',INT,required,,-
'user_id',INT,required,,-


,Values
Domain,
'gender',"'F', 'M'"


ExampleStatistics from StatisticsGen:
ExampleStatistics count=3


ExampleAnomalies from ExampleValidator:
ExampleAnomalies count=2
list=[id: 4
type_id: 20
uri: "/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/ExampleValidator/anomalies/4"
properties {
  key: "span"
  value {
    int_value: 0
  }
}
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_properties {
  key: "blessed"
  value {
    struct_value {
      fields {
        key: "eval"
        value {
          number_value: 1.0
        }
      }
      fields {
        key: "test"
        value {
          number_value: 1.0
        }
      }
      fields {
        key: "train"
        value {
          number_value: 1.0
        }
      }
    }
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "anomalies:2025-11-17T17:03:23.009550"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "ExampleValidator"
  }
}
custom_properties {
  key: "tfx_version"
  value {

ExampleStatistics from pre-transform stats of Transform:


### using TFDV to look at the transformed data

In [7]:

print("Schema from post-Transform:")
_list = store.get_artifacts_by_type("Schema")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_path = os.path.join(artifact_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(file_path)
#tfdv.visualize_artifacts(schema)
tfdv.display_schema(schema=schema)

print("ExampleStatistics from post-transform stats of Transform:")
_list = store.get_artifacts_by_type("ExampleStatistics")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    
print("ExampleAnomalies from post-transform of Transform:")
_list = store.get_artifacts_by_type("ExampleAnomalies")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_anomalies" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    anomalies = anomalies_pb2.Anomalies()
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)


Schema from post-Transform:
Schema count=3


,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,-
'gender',FLOAT,required,,-
'genres',FLOAT,required,,-
'hr',FLOAT,required,,-
'hr_wk',FLOAT,required,,-
'month',FLOAT,required,,-
'movie_id',FLOAT,required,,-
'occupation',FLOAT,required,,-
'rating',FLOAT,required,,-


ExampleStatistics from post-transform stats of Transform:


ExampleAnomalies from post-transform of Transform:


## Save the data schema with version control

In [8]:
import shutil

_list = store.get_artifacts_by_type("Schema")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
pre_transform_file_path = [os.path.join(artifact_uri, name) for name in os.listdir(artifact_uri)][0]

for artifact in _list:
    if "post_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
post_transform_file_path = [os.path.join(artifact_uri, name) for name in os.listdir(artifact_uri)][0]

pre_dir = os.path.join(get_project_dir(), "src/main/resources", "pre_transform")
post_dir = os.path.join(get_project_dir(), "src/main/resources", "post_transform")
os.makedirs(pre_dir, exist_ok=True)
os.makedirs(post_dir, exist_ok=True)

pre_schema_path = os.path.join(pre_dir, "schema.pbtxt")
post_schema_path = os.path.join(post_dir, "schema.pbtxt")

shutil.copyfile(pre_transform_file_path, pre_schema_path)
shutil.copyfile(post_transform_file_path, post_schema_path)


'/home/nichole/projects/github/recommender_systems/src/main/resources/post_transform/schema.pbtxt'

## Run baseline model pipeline with full dataset

In [9]:
pipeline_factory = PipelineComponentsFactory(
  num_examples=1000209, infiles_dict_ser=infiles_dict_ser,
  output_config_ser=output_config_ser, transform_dir=tr_dir,
  user_id_max=user_id_max, movie_id_max=movie_id_max,
  n_genres=n_genres, n_age_groups=n_age_groups,
  min_eval_size=MIN_EVAL_SIZE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, device="CPU",
  serving_model_dir=serving_model_dir, output_parquet_path=output_parquet_path)

SETUP_FILE_PATH = os.path.abspath('setup.py')
beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0',
  f'--setup_file={SETUP_FILE_PATH}',
  #f'--extra_package={ingest_tar_file}'
]

baseline_components = pipeline_factory.build_components(PIPELINE_TYPE.BASELINE,
  run_example_diff=False, pre_transform_schema_dir_path=pre_dir,
  post_transform_schema_dir_path=post_dir)

# create baseline model
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=baseline_components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)


INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.0001, 'conditions': [], 'values': [0.0001], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'regl2', 'default': 0.0, 'conditions': [], 'values': [0.0, 0.001, 0.01], 'ordered': True}}, {'class_name': 'Float', 'config': {'name': 'drop_rate', 'default': 0.5, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'embed_out_dim', 'default': 32, 'conditions': [], 'values': [32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'layer_sizes', 'default': '[32]', 'conditions': [], 'values': ['[32]'], 'ordered': False}}, {'class_name': 'Fixed', 'config': {'name': 'feature_acronym', 'conditions': [], 'value': ''}}, {'class_name': 'Fixed', 'config': {'name': 'incl_genres', 'conditions': [], 'value': True}}, {'class_name': 'Fixed

Trial 10 Complete [00h 00m 52s]
val_loss: 0.04324711859226227

Best val_loss So Far: 0.04225822165608406
Total elapsed time: 00h 08m 41s
Results summary
Results in /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Tuner/.system/executor_execution/14/.temp/14/movie_lens_2t_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 01 summary
Hyperparameters:
learning_rate: 0.0001
regl2: 0.001
drop_rate: 0.23609097545337843
embed_out_dim: 32
layer_sizes: [32]
feature_acronym: 
incl_genres: True
BATCH_SIZE: 64
NUM_EPOCHS: 20
use_bias_corr: False
user_id_max: 6040
movie_id_max: 3952
n_age_groups: 7
n_genres: 18
run_eagerly: False
device: CPU
MAX_TUNE_TRIALS: 10
input_dataset_element_spec_ser: gASV1AIAAAAAAAB9lCiMA2FnZZSMInRlbnNvcmZsb3cucHl0aG9uLmZyYW1ld29yay50ZW5zb3KUjApUZW5zb3JTcGVjlJOUjCh0ZW5zb3JmbG93LnB5dGhvbi5mcmFtZXdvcmsudGVuc29yX3NoYXBllIwLVGVuc29yU2hhcGWUk5RdlChoBYwJRGltZW5zaW9ulJOUS0CFlFKUaApLAYWUUpRlhZRSlIwidGVuc29yZmxvdy5we

INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_trainer-0.0+fb13acbbc55a3dc5e4140287ddbb8a25fbbb448ebb0d37bb248f454000152047-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:HyperParameters for training: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.0001, 'conditions': [], 'values': [0.0001], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'regl2', 'default': 0.0, 'conditions': [], 'values': [0.0, 0.001, 0.01], 'ordered': True}}, {'class_name': 'Float', 'config': {'name': 'drop_rate', 'default': 0.5, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'embed_out_dim', 'default': 32, 'conditions': [], 'values': [32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'layer_sizes', 'default': '[32]', 'conditions': [], 'values': ['[32]'], 'ord

INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
dim {
  size: 18
}
. Setting to DenseTensor.
INFO:absl:Feature hr has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature hr_wk has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sec_into_yr has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature weekday has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:

INFO:absl:Model: "two_tower_dnn_1"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ query_model_1 (QueryModel)      │ (None, 32)             │       194,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ candidate_model_1               │ (None, 32)             │       129,216 │
│ (CandidateModel)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dot_1 (Dot)                     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 323,616 (1.23 MB)
 Trainab

...

INFO:absl:eval_transformed_features = {'yr': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:12' shape=(None, 1) dtype=float32>, 'gender': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:1' shape=(None, 1) dtype=float32>, 'user_id': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:10' shape=(None, 1) dtype=float32>, 'hr_wk': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:4' shape=(None, 1) dtype=float32>, 'month': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:5' shape=(None, 1) dtype=float32>, 'movie_id': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:6' shape=(None, 1) dtype=float32>, 'occupation': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:7' shape=(None, 1) dtype=float32>, 'sec_into_yr': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:9' shape=(None, 1) dtype=float32>, 'rating': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:8' shape=(None, 1) dtype=float32>,

INFO:tensorflow:Assets written to: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/15/Format-Serving/assets


INFO:absl:Writing fingerprint to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/15/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/15/Format-Serving. ModelRun written to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model_run/15
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 15 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Deleted stateful_working_dir /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/.system/stateful_working_dir/c7f9c620-0e26-4452-9723-b9248a710ec4
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model_run/15"
, art

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Blessing result True written to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Evaluator/blessing/16.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 16 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Deleted stateful_working_dir /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Evaluator/.system/stateful_working_dir/d9ad62e7-ac61-46d3-8ad7-60b0da77fa51
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Evaluator/blessing/16"
, artifact_type: name: "ModelBlessing"
)], 'evaluation': [Artifact(artifact: uri: "/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Evaluator/evaluation/16"
, artifact_type: name: "ModelEvaluation"
)]}) for execution 16
INFO:absl:MetadataStore with DB connection initializ

### Visualize Baseline model fit

In [10]:
from tensorflow_model_analysis.proto import validation_result_pb2

def print_model_evaluation(artifact_uri):

    for name in os.listdir(artifact_uri):
        print(f"file_name={name}")
        file_path = os.path.join(artifact_uri, name)
        if name.startswith("metrics"):
            raw_dataset = tf.data.TFRecordDataset(file_path)
            for serialized_proto in raw_dataset:
              metrics_for_slice = metrics_for_slice_pb2.MetricsForSlice()
              metrics_for_slice.ParseFromString(serialized_proto.numpy())
              print(f'metrics_for_slice={metrics_for_slice}')
        elif name.startswith("validations"):
            raw_dataset = tf.data.TFRecordDataset([file_path])
            for serialized_validation_result in raw_dataset.as_numpy_iterator():
                validation_result = validation_result_pb2.ValidationResult()
                validation_result.ParseFromString(serialized_validation_result)
                
                # Now you can inspect the validation_result object
                print(f"Validation OK: {validation_result.validation_ok}")
                
                # If validation_ok is False, you can see the failing checks:
                if not validation_result.validation_ok:
                    if hasattr(validation_result, "metric_failures"):
                        for failure in validation_result.metric_failures:
                            print(f"  Failed Metric: {failure.metric_key.name}")
                            print(f"  Failure Type: {failure.failure_type}")
                            print(f"  Observed Value: {failure.observed_value.value}")
            validation_result = tfma.load_validation_result(file_path)
            print(validation_result)


In [11]:
from tensorflow_model_analysis.proto import metrics_for_slice_pb2

#add training and eval plots: Trainer/model_run
#  Tensorboard can visualize them
print(f'plot ModelRun using tensorboard')
%load_ext tensorboard
_list = store.get_artifacts_by_type("ModelRun")
print(f'ModelRun count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "Trainer" in artifact.uri:
        artifact_uri = artifact.uri
        %tensorboard --logdir {artifact_uri}

#no baseline model yet, so just read files:
print("ModelEvaluation from Evaluator:")
_list = store.get_artifacts_by_type("ModelEvaluation")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
artifact_uri = _list[0].uri
assert(artifact_uri is not None)
#file_path = os.path.join(artifact_uri, "metrics*")
print(f'ModelEvaluation uri={artifact_uri}')
print_model_evaluation(artifact_uri)


plot ModelRun using tensorboard
ModelRun count=1


ModelEvaluation from Evaluator:
Schema count=1
ModelEvaluation uri=/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Evaluator/evaluation/16
file_name=metrics-00000-of-00001.tfrecord
metrics_for_slice=slice_key {
}
metric_keys_and_values {
  key {
    name: "example_count"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 99930.0
    }
  }
}

file_name=eval_config.json
file_name=plots-00000-of-00001.tfrecord
file_name=attributions-00000-of-00001.tfrecord
file_name=validations.tfrecord
Validation OK: True
validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}



2025-11-17 17:35:12.672056: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-17 17:35:12.691778: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Run full model pipeline with full dataset

In [12]:
artifact_types = store.get_artifact_types()
logging.debug(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
logging.debug(f"MLMD store artifacts={artifacts}")

components = pipeline_factory.build_components(PIPELINE_TYPE.PRODUCTION,
  run_example_diff=False, pre_transform_schema_dir_path=pre_dir,
  post_transform_schema_dir_path=post_dir)
  
# simulate experimentation of one model family
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)


INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.0001, 'conditions': [], 'values': [0.0001], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'regl2', 'default': 0.0, 'conditions': [], 'values': [0.0, 0.001, 0.01], 'ordered': True}}, {'class_name': 'Float', 'config': {'name': 'drop_rate', 'default': 0.5, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'embed_out_dim', 'default': 32, 'conditions': [], 'values': [32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'layer_sizes', 'default': '[32]', 'conditions': [], 'values': ['[32]'], 'ordered': False}}, {'class_name': 'Fixed', 'config': {'name': 'feature_acronym', 'conditions': [], 'value': 'a'}}, {'class_name': 'Fixed', 'config': {'name': 'incl_genres', 'conditions': [], 'value': True}}, {'class_name': 'Fixe

Trial 10 Complete [00h 00m 56s]
val_loss: 0.04251469299197197

Best val_loss So Far: 0.04244314506649971
Total elapsed time: 00h 09m 28s
Results summary
Results in /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Tuner/.system/executor_execution/24/.temp/24/movie_lens_2t_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 02 summary
Hyperparameters:
learning_rate: 0.0001
regl2: 0.001
drop_rate: 0.1513757562146236
embed_out_dim: 32
layer_sizes: [32]
feature_acronym: a
incl_genres: True
BATCH_SIZE: 64
NUM_EPOCHS: 20
use_bias_corr: False
user_id_max: 6040
movie_id_max: 3952
n_age_groups: 7
n_genres: 18
run_eagerly: False
device: CPU
MAX_TUNE_TRIALS: 10
input_dataset_element_spec_ser: gASV1AIAAAAAAAB9lCiMA2FnZZSMInRlbnNvcmZsb3cucHl0aG9uLmZyYW1ld29yay50ZW5zb3KUjApUZW5zb3JTcGVjlJOUjCh0ZW5zb3JmbG93LnB5dGhvbi5mcmFtZXdvcmsudGVuc29yX3NoYXBllIwLVGVuc29yU2hhcGWUk5RdlChoBYwJRGltZW5zaW9ulJOUS0CFlFKUaApLAYWUUpRlhZRSlIwidGVuc29yZmxvdy5we

INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_trainer-0.0+661ad9f4c7361ec1fcd2930ddce63297061894550000fc9d78347d8466ab6018-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:HyperParameters for training: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.0001, 'conditions': [], 'values': [0.0001], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'regl2', 'default': 0.0, 'conditions': [], 'values': [0.0, 0.001, 0.01], 'ordered': True}}, {'class_name': 'Float', 'config': {'name': 'drop_rate', 'default': 0.5, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'embed_out_dim', 'default': 32, 'conditions': [], 'values': [32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'layer_sizes', 'default': '[32]', 'conditions': [], 'values': ['[32]'], 'ord

INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
dim {
  size: 18
}
. Setting to DenseTensor.
INFO:absl:Feature hr has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature hr_wk has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sec_into_yr has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature weekday has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:

INFO:absl:Model: "two_tower_dnn_1"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ query_model_1 (QueryModel)      │ (None, 32)             │       195,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ candidate_model_1               │ (None, 32)             │       129,216 │
│ (CandidateModel)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dot_1 (Dot)                     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 324,928 (1.24 MB)
 Trainab

...

INFO:absl:Function `call` contains input name(s) resource with unsupported characters which will be renamed to candidate_model_1_1_dense_candidate_1_dense_5_1_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Function `serve_query_model` contains input name(s) resource with unsupported characters which will be renamed to query_model_1_1_dense_query_1_dense_3_1_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Function `serve_candidate_model` contains input name(s) resource with unsupported characters which will be renamed to candidate_model_1_1_dense_candidate_1_dense_5_1_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupat

transform_features_fn spec = {raw_feature_spec}


INFO:absl:Function `transform_features_fn` contains input name(s) 4030815, 4030819, 4030823 with unsupported characters which will be renamed to transform_features_layer_4030815, transform_features_layer_4030819, transform_features_layer_4030823 in the SavedModel.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) 4030931, 403093

INFO:tensorflow:Assets written to: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/25/Format-Serving/assets


INFO:absl:Writing fingerprint to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/25/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model/25/Format-Serving. ModelRun written to /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model_run/25
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 25 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Deleted stateful_working_dir /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/.system/stateful_working_dir/c7e2b09f-f04e-4984-994a-2e3b038f2e0f
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/Trainer/model_run/25"
, art

### Visualize model fits

In [13]:
#add training and eval plots: Trainer/model_run
#  Tensorboard can visualize them
print(f'plot ModelRun using tensorboard')
%load_ext tensorboard
_list = store.get_artifacts_by_type("ModelRun")
print(f'ModelRun count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "Trainer" in artifact.uri:
        artifact_uri = artifact.uri
        %tensorboard --logdir {artifact_uri}

print("ModelEvaluation from Evaluator:")
_list = store.get_artifacts_by_type("ModelEvaluation")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
artifact_uri = _list[0].uri
print_model_evaluation(artifact_uri)


plot ModelRun using tensorboard
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
ModelRun count=2


Reusing TensorBoard on port 6006 (pid 567812), started 0:18:48 ago. (Use '!kill 567812' to kill it.)

ModelEvaluation from Evaluator:
Schema count=2
file_name=metrics-00000-of-00001.tfrecord
metrics_for_slice=slice_key {
}
metric_keys_and_values {
  key {
    name: "example_count"
    model_name: "candidate"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 99930.0
    }
  }
}
metric_keys_and_values {
  key {
    name: "example_count"
    model_name: "baseline"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 99930.0
    }
  }
}
metric_keys_and_values {
  key {
    name: "mean_absolute_error"
    model_name: "candidate"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 0.16679825215227348
    }
  }
}
metric_keys_and_values {
  key {
    name: "mean_absolute_error"
    model_name: "baseline"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 0.16516457820218725
    }
  }
}
metric_keys_and_values {
  key {
    name: "example_count"
    model_name: "candidate"
    is_diff: true
    example_

2025-11-17 17:54:00.245482: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-17 17:54:00.277250: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Monitoring the deployed model

The model deployed to production can be monitored to detect when the business utility drops below an acceptable threshold.
The business utility here is linked to the accuracy of recommending movies to the user that they would like and haven't seen.

Detecting model degradation when users have watched a movie, and before they have rated them, requires bootstrapping
using an extra model trained from reference data that has the user ratings. 

NannyML is a popular library used for detecting changes in ML model features, targets, and relationships between
them.  
The (NannyML Model performance estimator) [https://nannyml.readthedocs.io/en/stable/how_it_works/performance_estimation.html#dle-implementation-details] for regression  is their Direct Loss Estimator algorithm (DLE).

The models:
* f = the monitored recommender model
* h = the Nanny model (discussed below)

The datasets used:
* train: used to train f.  contains features X and targets Y where targets are the ratings.
* reference: used to train h. contains features X and targets Y
* analysis: recent data.  contains only features X
The datasets are ordered by time.

The nanny model h is an extra mode used for degradation monitoring.
* h is trained with X_reference, and y_reference
    * for each observation in X, calculate absolute error AE_reference = | f(X_reference) - y_reference |
    * train A_reference = h(X_reference, f(X_reference))
          * DLE for regression uses LightGBM
* predict AE_analysis using h
    * for each observation in X: AE_analysis_est = h( X_analysis, f(X_analysis))
    * MAE_analysis_est = mean of the AE_analysis_est
* the variance of large chunks of data are determined and the upper and lower bounds for anomalies are set using the sample error using those variances.  When a predicted MAE exceed the thresholds, an alert can be triggered and root cause analysis can be performed to decide if the model should be retrained or refactored and retrained as part of a Continuous Training (CT) workflow.

DLE assumes:
* no concept drift
* no covariate shift to previously unseen regions of input space
* noise is heteroscedastic around the monitored model target, y,
  and it is dependent on the monitored model input features
* the dataset to be analyzed is large enough so that the performance estimate is not dominated by sampling error

#### Model monitoring is a separate process from the serving inference, so an application can be built for it or cloud options
can be used when configuring batch inference.

TODO: add cloud model monitoring options and details...
Apache Airflow,  Vertex AI Model Monitoring, Azure ML Model monitoring though NannyML is an option through Azure Marketplace, Amazon SageMaker Model monitor and NannyML is an option through the marketplace,

## Linking to business value
TODO: add the ranking model and precision@K, recall@K and NDCG.  this is in the other notebook, so can be merged when finished....

TODO: list supplemental data that could be used to link to business value:
click through rate, conversion rate, user engagement, churn, revenue uplift from associated ads.

TODO: quantifying the business value:
A/B testing with control group and test group where the later receives ranked recommendation from the new model.


## Summary
TFX pipelines are valuable tools in MLOps
* highly scalable
    * distributed data pre-processing
    * distributed model training
* configurable to run on a variety of architectures
    * e.g. TPUs, GPUs for accelerated training
* configurable for variety of different data source types
* configurable for fault tolerance and failover
* configurable for different types of pipeline storage and model registry
* configurable for cloud native, cloud specific, and local environments
* robust, consistent processing
* configurable exported model types compatible with keras, pytorch, jax:
    * default saved_models
    * liteRT for embedded and IOT environments
    * TensorflowJS for browsers
* usable in cloud environments
    * compatible with many orchestrators
        * Apache Airflow, Apache Beam, Kubeflow pipelines
    * can be used in containers (required by kubernetes for example)
    * compatible with cluster management systems like kubernetes, flink, spark, and cloud native offerings
    * can be compiled into Kubeflow pipelines
    * can be run from a command line interface in the cloud or locally
* the MLMD store is usable by a variety of monitoring systems including cloud-specific systems
* Tensorboard for visualization of model training output, TFDV for visualization of data
* compatible with use of BI tools like GCP's Looker, DataPrep
* inter-operative with Vertex AI Training, Prediction, Dataflow